# Module importing

In [0]:
import pandas as pd
import numpy as np

In [0]:
import tqdm

In [0]:
import re

In [0]:
from bs4 import BeautifulSoup

# Data loading

In [9]:
import os
os.environ['KAGGLE_USERNAME'] = "kryvokhyzha" # username from the json file
os.environ['KAGGLE_KEY'] = "b30f0451d14f9a4f8a939d2fbadf03e8" # key from the json file
!kaggle competitions download -c company-acceptance-prediction # api copied from kaggle

 81% 67.0M/82.3M [00:00<00:00, 77.1MB/s]
100% 82.3M/82.3M [00:00<00:00, 152MB/s] 
 98% 177M/180M [00:02<00:00, 121MB/s]
100% 180M/180M [00:02<00:00, 70.5MB/s]
  0% 0.00/167k [00:00<?, ?B/s]
100% 167k/167k [00:00<00:00, 170MB/s]


# Data serialization

In [0]:
train_df = pd.read_pickle('train_df.pkl.zip')

In [0]:
test_df = pd.read_pickle('test_df.pkl.zip')

In [197]:
test_df.head(5)

,id,html,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product
0,071c2e15-5fd0-4e7d-b78d-26b7c34491e1,"[b'<!DOCTYPE html>\r\n<html id=""ctl00_htmlMast...",[We use cookies to tailor our service and ensu...,"[technologies, descriptive, mapping, learning,...","IT support, IT managed services, IT integratio...","manufacture, production, processing, distribut...",,
1,e534df73-b385-4a12-8368-38992d3af7d3,"[b'\n<!DOCTYPE html>\n\n<html lang=""en-US"">\n\...",[About us\nAbout Powernet\nPartnerships\nCusto...,"[outsourcing, projects, IT, sensors, convergen...","software development, database development, sy...","wholesale, distribution, retail selling, manuf...",,
2,4e38c9ff-bffe-4251-b2b2-94777c7344fe,[b'<!--\n.----------------. .----------------...,[Work\nApproach\nPeople\nValues\nBlog\nCareers...,"[Inventing, products, risk, development, servi...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,
3,a7f8870c-30f8-4851-8f8b-4c16942b397f,[b'<!DOCTYPE html>\n<!--[if lt IE 7]><html cla...,[Menu\nMenu\nSupport\nInfrastructure\nColocati...,"[low, chain, portfolio, services, Procurement,...","Investment research and information services, ...","Construction, production and supply, Advertisi...",,
4,c062f411-ae3e-486a-9cba-2267889adb2b,"[b'<!DOCTYPE html><html lang=""en-US""><head><me...",[Français (French)\nEnglish (English)\n日本語 (Ja...,"[recruitment, producing, Evaluation, Payroll, ...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,


# Data preprocessing

## Dropping byte-like parts

In [0]:
# :(

## Extract comments

In [0]:
# :(

## Extract meta information

In [0]:
train_df['Title'] = np.nan
train_df['Keyword'] = np.nan
train_df['Locale'] = np.nan
train_df['Description'] = np.nan

In [315]:
for idx, pages in tqdm.tqdm(enumerate(train_df.html)):
  title_res = []
  keywords_res = []
  locale_res = []
  description_res = []
  for page in pages:
    soup = BeautifulSoup(page, "lxml")  

    res = ''
    for tag in soup.find_all("meta", property='og:title'):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      title_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", attrs={'name':'keywords'}):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      keywords_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", property='og:locale'):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' ' 
      except:
        pass
    if res != '':
      locale_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", attrs={'name':'description'}):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      description_res.append(res.strip()) 


  train_df.loc[idx, 'Title'] = str(title_res) if title_res != [] else np.nan
  train_df.loc[idx, 'Keyword'] = str(keywords_res) if keywords_res != [] else np.nan
  train_df.loc[idx, 'Locale'] = str(locale_res) if locale_res != [] else np.nan
  train_df.loc[idx, 'Description'] = str(description_res) if description_res != [] else np.nan


15515it [35:49,  7.22it/s]


### Locale preprocessing

In [0]:
# sl - seldom, ot - other
train_df.Locale = train_df.Locale.fillna('["ot').apply(lambda x: x[2:4]).map({'ot': 'ot', 'en': 'en', 'ja': 'ja', 'pl':'pl',
                                                                              'es': 'sl', 'de': 'sl', 'it': 'sl',
                                                                              'fr': 'sl', 'bg': 'sl', 'zh': 'sl',
                                                                              'sv': 'sl','nl': 'sl','fi': 'sl',
                                                                              'nb': 'sl'})

### Title preprocessing

In [0]:
train_df.Title = train_df.Title.fillna('without_title').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

### Keyword preprocessing

In [0]:
train_df.Keyword = train_df.Keyword.fillna('without_keyword').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

### Description preprocessing

In [357]:
train_df.Description[95]

'manufacturing and commercializing single use medical devices for health care professionals raffin medical has been developing since 1914 raffin medical is a manufacturer and a distributor of singleuse medical devices for health care professionals'

In [0]:
train_df.Description = train_df.Description.fillna('without_description').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

In [0]:
train_df.Description = train_df.Description.apply(lambda x: x if re.sub(r'[a-zA-Z0-9\s\-_]+', '', x.strip()) == '' else 'wrong_description')

## Keywords (model base) preproccesing

In [0]:
train_df.keywords = train_df.keywords.apply(lambda x: [re.sub(r'[^\w\s\d]+', ' ', i).lower() for i in x] if x != [] else ['empty_keyword_list'])

## Accepted/rejected preproccesing

In [0]:
train_df.accepted_function = train_df.accepted_function.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.rejected_function = train_df.rejected_function.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.accepted_product = train_df.accepted_product.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.rejected_product = train_df.rejected_product.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

## Replace NaN like values

In [0]:
train_df = train_df.replace(['', '_nan_'], [np.nan, np.nan])

## Filling keyword and description

In [0]:
train_df.Keyword = train_df.Keyword.fillna('without_keyword')

In [0]:
train_df.Description = train_df.Description.fillna('without_description')

## Combine all texts and html pages

In [0]:
# :(

# EDA

In [469]:
print(train_df.text[15][0]) #1439

This page is being redirected. If it does not load, click https://www.fscables.com/featured_page/antennax--the-ultra-low-loss-coaxial-cable-range.html



In [464]:
print(str(train_df.html[2028][0]))

b"<br />\n<b>Fatal error</b>:  Uncaught Error: Call to undefined function mcrypt_create_iv() in /home/forumtna/public_html/system/library/encryption.php:8\nStack trace:\n#0 /home/forumtna/public_html/index.php(215): Encryption-&gt;__construct('8751d0dc1cb676e...')\n#1 {main}\n  thrown in <b>/home/forumtna/public_html/system/library/encryption.php</b> on line <b>8</b><br />\n"


In [11]:
train_df['target'].value_counts() / train_df['target'].value_counts().sum()

1    0.744892
2    0.190139
0    0.064969
Name: target, dtype: float64

# Feature generation

In [0]:
train_df['html_page_cnt'] = train_df.html.apply(lambda x: len(x)).astype(np.int16)

## Mean number of rows in texts

In [0]:
train_df['text_rows_mean'] = train_df.text.apply(lambda x: np.sum([len(i.split('\n')) for i in x]) // len(x)).astype(np.int16)

## Missing values columns

In [0]:
train_df['is_mis_accepted_function'] = pd.isnull(train_df.accepted_function).map({True:1, False:0})

In [0]:
train_df['is_mis_rejected_function'] = pd.isnull(train_df.rejected_function).map({True:1, False:0})

In [0]:
train_df['is_mis_accepted_product'] = pd.isnull(train_df.accepted_product).map({True:1, False:0})

In [0]:
train_df['is_mis_rejected_product'] = pd.isnull(train_df.rejected_product).map({True:1, False:0})

In [0]:
train_df['is_mis_keyword'] = train_df['keywords'].apply(lambda x: 1 if x == ['empty_keyword_list'] else 0)

In [0]:
train_df = train_df.fillna('is_missing')

## Redirect feature

In [0]:
train_df['have_error'] = train_df.text.apply(lambda x: 1 if (('redirected' in x[0].lower() or 'error' in x[0].lower()) and len(x) == 1) else 0)

## Similarity criteria features (in accepted and rejected)

In [0]:
acc_f_rej_f_sim = []
for row in train_df.itertuples():
  accepted_f = set(row.accepted_function.replace(',', '').split(' '))
  rejected_f = set(row.rejected_function.replace(',', '').split(' '))
  acc_f_rej_f_sim.append(len(accepted_f.intersection(rejected_f)) / len(accepted_f.union(rejected_f)))

train_df['acc_f_rej_f_sim'] = acc_f_rej_f_sim

In [0]:
acc_p_rej_p_sim = []
for row in train_df.itertuples():
  accepted_p = set(row.accepted_product.replace(',', '').split(' '))
  rejected_p = set(row.rejected_product.replace(',', '').split(' '))
  acc_p_rej_p_sim.append(len(accepted_p.intersection(rejected_p)) / len(accepted_p.union(rejected_p)))

train_df['acc_p_rej_p_sim'] = acc_p_rej_p_sim

In [0]:
kwd_acc_p_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  accepted_p = set(row.accepted_product.replace(',', '').split(' '))
  kwd_acc_p_sim.append(len(kwd.intersection(accepted_p)) / len(kwd.union(accepted_p)))

train_df['kwd_acc_p_sim'] = kwd_acc_p_sim

In [0]:
kwd_rej_p_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  rejected_p = set(row.rejected_product.replace(',', '').split(' '))
  kwd_rej_p_sim.append(len(kwd.intersection(rejected_p)) / len(kwd.union(rejected_p)))

train_df['kwd_rej_p_sim'] = kwd_rej_p_sim

In [0]:
kwd_acc_f_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  accepted_f = set(row.accepted_function.replace(',', '').split(' '))
  kwd_acc_f_sim.append(len(kwd.intersection(accepted_f)) / len(kwd.union(accepted_f)))

train_df['kwd_acc_f_sim'] = kwd_acc_f_sim

In [0]:
kwd_rej_f_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  rejected_f = set(row.rejected_function.replace(',', '').split(' '))
  kwd_rej_f_sim.append(len(kwd.intersection(rejected_f)) / len(kwd.union(rejected_f)))

train_df['kwd_rej_f_sim'] = kwd_rej_f_sim

In [0]:
train_df['dif_kwd_acc_p_rej_p'] = train_df['kwd_acc_p_sim'] - train_df['kwd_rej_p_sim']
train_df['dif_kwd_acc_j_rej_j'] = train_df['kwd_acc_f_sim'] - train_df['kwd_rej_f_sim']

## All accepted and rejected fraction

In [0]:
all_accepted_function = set()
all_rejected_function = set()

all_accepted_product = set()
all_rejected_product = set()

In [0]:
for row in train_df.itertuples():
  all_accepted_function = all_accepted_function.union(set(row.accepted_function.split(', ')))
  all_rejected_function = all_rejected_function.union(set(row.rejected_function.split(', ')))
  all_accepted_product = all_accepted_product.union(set(row.accepted_product.split(', ')))
  all_rejected_product = all_rejected_product.union(set(row.rejected_product.split(', ')))

In [0]:
frc_acc_f = []
frc_rej_f = []
frc_acc_p = []
frc_rej_p = []

for row in train_df.itertuples():
  frc_acc_f.append(len(set(row.accepted_function.split(', '))) / len(all_accepted_function))
  frc_rej_f.append(len(set(row.rejected_function.split(', '))) / len(all_rejected_function))

  frc_acc_p.append(len(set(row.accepted_product.split(', '))) / len(all_accepted_product))
  frc_rej_p.append(len(set(row.rejected_product.split(', '))) / len(all_rejected_product))

train_df['frc_acc_f'] = frc_acc_f
train_df['frc_rej_f'] = frc_rej_f
train_df['frc_acc_p'] = frc_acc_p
train_df['frc_rej_p'] = frc_rej_p

In [533]:
train_df

,id,html,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product,target,Title,Keyword,Locale,Description,html_page_cnt,text_rows_mean,is_mis_accepted_function,is_mis_rejected_function,is_mis_accepted_product,is_mis_rejected_product,have_error,acc_f_rej_f_sim,acc_p_rej_p_sim,is_mis_keyword,kwd_acc_p_sim,kwd_rej_p_sim,kwd_acc_f_sim,kwd_rej_f_sim,dif_kwd_acc_p_rej_p,dif_kwd_acc_j_rej_j,frc_acc_f,frc_rej_f,frc_acc_p,frc_rej_p
0,b811ea46-1bc5-4a18-a64c-7504c382da44,"[b'<!DOCTYPE html>\n<html lang=""en"" class=""pre...",[Release\n2020\n2019\n2018\n2017\n2020\n2019\n...,"[tools, services, projects, incubates, investm...","accounting, bookeeping, auditing, finance, hr,...","manufacture production processing assembly, re...",is_missing,is_missing,2,about dream incubator di company di,without_keyword,ot,wrong_description,2,73,0,0,1,1,0,0.000000,1.000000,0,0.000000,0.00,0.047619,0.044776,0.000000,0.002843,0.018265,0.011785,0.002597,0.004149
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,"[b'<!DOCTYPE html>\n<html lang=""en-US"">\n <he...",[SAP Integrated Business Planning (IBP)\nSAP A...,"[budgeting, decision, cycles, closing, simplif...","it consultancy, it support services, it manage...","distribution, sale, trade, wholesale, retail s...",is_missing,is_missing,2,sap bpc business planning and consolidation s...,without_keyword,en,forecasting and consolidation processes of you...,1,48,0,0,1,1,0,0.061224,1.000000,0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.019787,0.023569,0.002597,0.004149
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,"[b'<!doctype html>\n<html lang=""en-US"">\n<head...",[facebook\ntwitter\nlinkedin\ninstagram\nWelco...,[empty_keyword_list],"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",is_missing,is_missing,1,home ams media group cookie policy ams media...,without_keyword,en,a fully accredited independent media planning ...,2,72,0,0,1,1,0,0.117647,1.000000,1,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.031963,0.045455,0.002597,0.004149
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,"[b'<?xml version=""1.0"" encoding=""iso-8859-1""?>...",[ANDERSON BUSINESS TRAVEL\nSpecialist Independ...,"[provide, service, financial, arranging, trave...","insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",is_missing,is_missing,2,without_title,without_keyword,ot,without_description,2,11,0,0,1,1,0,0.024390,1.000000,0,0.000000,0.00,0.083333,0.000000,0.000000,0.083333,0.015221,0.023569,0.002597,0.004149
4,74a95f04-e440-4af0-850a-eec5a44ba397,[b'\r\n\r\n<!DOCTYPE html>\r\n\r\n<html xmlns=...,[Amari Ireland Ltd\nStandards\nPeople\nQuality...,"[independent, centre, forms, copper, bronze, s...","sales agent and sales representative, product ...","manufacture, production, processing and retail...","product, motor vehicles, buses and trucks, mot...",is_missing,1,without_title,uk metals suppliers marine aluminium stainless...,ot,without_description,2,40,0,0,0,1,0,0.041667,0.000000,0,0.064516,0.00,0.018519,0.018868,0.064516,-0.000349,0.010654,0.010101,0.036364,0.004149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15510,0ea4fb41-d5d8-4086-aceb-ced6665562c1,[b'<!DOCTYPE html>\n<html>\n<head>\n<meta http...,[About Thule Group\nStrategic building blocks\...,"[enthusiasts, duffel, chain, reports, industry...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",is_missing,is_missing,1,about thule group what we are about,without_keyword,ot,simplified captures the ambition of how we dev...,2,135,0,0,1,1,0,0.087719,1.000000,0,0.000000,0.00,0.025974,0.021505,0.000000,0.004469,0.022831,0.042088,0.002597,0.004149
15511,7563c3e1-0ab0-415a-b565-b8f71e097014,[b'\xef\xbb\xbf<!doctype html>\r\n<html>\r\n<h...,[Solutions\nEnterprise\nBank\nConstruction\nSc...,[empty_keyword_list],"accounting, bookee

In [1]:
train_df

NameError: ignored